In [2]:
# PE算法填补缺失值
import numpy as np

In [2]:
# 创建用均值填充的初始值矩阵
matrix = np.array([[6.0, 0, 3],
                    [7, 2, 6],
                    [5, 1, 2],
                    [6.0, 1.0, 5]])

In [3]:
# 按列求均值
mean_values = np.mean(matrix, axis=0)

In [4]:
# 按列计算协方差矩阵
cov_matrix = np.cov(matrix, rowvar=False, ddof=0)  #默认情况下，ddof等于1，即使用 n-1作为除数。小样本条件下，将ddof设置为0，使用 n作为除数。

In [5]:
#对于缺失值x11做估计
x11_pred = mean_values[0] + cov_matrix[0, 1:]@np.linalg.inv(cov_matrix[1:, 1:])@(matrix[0,1:] - mean_values[1:]).T
x11_pred

5.7272727272727275

In [6]:
#与x11有关的协方差估计
x11_2_pred_array = cov_matrix[0,0] - cov_matrix[0, 1:]@np.linalg.inv(cov_matrix[1:, 1:])@cov_matrix[:1, 1:].T + pow(x11_pred , 2)
x11_2_pred = x11_2_pred_array[0]
x11_x12_pred = x11_pred * matrix[0,1]
x11_x13_pred = x11_pred * matrix[0,2]

In [7]:
#对缺失值（x41,x42）的向量估计  
pred_array_x41_x42 =  mean_values[0:2] + cov_matrix[2:, 0:2]*np.linalg.inv(cov_matrix[2:, 2:])*(matrix[3,2] - mean_values[2])
x41_pred = pred_array_x41_x42[0,0]
x42_pred = pred_array_x41_x42[0,1]

In [8]:
b = np.array([[x41_pred],[x42_pred]])

In [9]:
#与x41，x42有关的协方差矩阵估计
pred_matrix_x41_x42 = cov_matrix[:2,:2] - cov_matrix[2:, 0:2].T*np.linalg.inv(cov_matrix[2:, 2:])@cov_matrix[2:, 0:2] + b@b.T
x41_x43_pred = x41_pred*matrix[3, 2]
x42_x43_pred = x42_pred*matrix[3, 2]
x41_2_pred = pred_matrix_x41_x42[0,0]
x42_2_pred = pred_matrix_x41_x42[1,1]
x41_x42_pred = pred_matrix_x41_x42[0,1]

In [10]:
#更新均值和协方差矩阵
mean_new = 1/4 * np.array([[x11_pred  + matrix[1,0] + matrix[2,0] +   x41_pred],
                          [matrix[0,1] + matrix[1,1] + matrix[2,1] +   x42_pred],
                          [matrix[0,2] + matrix[1,2] + matrix[2,2] + matrix[3,2]]])

In [11]:
T = np.zeros((3,3))
T[0,0] = x11_2_pred + pow(matrix[1,0],2) + pow(matrix[2,0],2) + x41_2_pred
T[0,1] = x11_x12_pred + matrix[1,0]*matrix[1,1] + matrix[2,0]*matrix[2,1] + x41_x42_pred
T[0,2] = x11_x13_pred + matrix[1,0]*matrix[1,2] + matrix[2,0]*matrix[2,2] + x41_x43_pred
T[1,0] = T[0,1]
T[1,1] = pow(matrix[0,1],2) + pow(matrix[1,1],2)+ pow(matrix[2,1],2) + x42_2_pred
T[1,2] = matrix[0,1]*matrix[0,2] + matrix[1,1]*matrix[1,2]+ matrix[2,1]*matrix[2,2] +x42_x43_pred
T[2,0] = T[0,2]
T[2,1] = T[1,2]
T[2,2] = pow(matrix[0,2],2) + pow(matrix[1,2],2) + pow(matrix[2,2],2) + pow(matrix[3,2],2)

In [12]:
cov_new = 1/4*T - mean_new@mean_new.T
cov_new

array([[0.60530992, 0.33329545, 1.16818182],
       [0.33329545, 0.585625  , 0.825     ],
       [1.16818182, 0.825     , 2.5       ]])

In [13]:
m =1

In [14]:
while np.linalg.norm(cov_new - cov_matrix)  >0.00001 or np.linalg.norm(mean_new - mean_values.reshape(-1, 1)) >0.00001 and m <100:
    mean_values = mean_new.flatten()
    cov_matrix = cov_new
    #对于缺失值x11做估计
    x11_pred = mean_values[0] + cov_matrix[0, 1:]@np.linalg.inv(cov_matrix[1:, 1:])@(matrix[0,1:] - mean_values[1:])
    #与x11有关的协方差估计
    x11_2_pred_array = cov_matrix[0,0] - cov_matrix[0, 1:]@np.linalg.inv(cov_matrix[1:, 1:])@cov_matrix[:1, 1:].T + pow(x11_pred , 2)
    x11_2_pred = x11_2_pred_array[0]
    x11_x12_pred = x11_pred * matrix[0,1]
    x11_x13_pred = x11_pred * matrix[0,2]
    #对缺失值（x41,x42）的向量估计  
    pred_array_x41_x42 =  mean_values[0:2] + cov_matrix[2:, 0:2]*np.linalg.inv(cov_matrix[2:, 2:])*(matrix[3,2] - mean_values[2])
    x41_pred = pred_array_x41_x42[0,0]
    x42_pred = pred_array_x41_x42[0,1]
    b = np.array([[x41_pred],[x42_pred]])
    #与x41，x42有关的协方差矩阵估计
    pred_matrix_x41_x42 = cov_matrix[:2,:2] - cov_matrix[2:, 0:2].T*np.linalg.inv(cov_matrix[2:, 2:])@cov_matrix[2:, 0:2] + b@b.T
    x41_x43_pred = x41_pred*matrix[3, 2]
    x42_x43_pred = x42_pred*matrix[3, 2]
    x41_2_pred   = pred_matrix_x41_x42[0,0]
    x42_2_pred   = pred_matrix_x41_x42[1,1]
    x41_x42_pred = pred_matrix_x41_x42[0,1]
    #更新均值和协方差矩阵
    mean_new = 1/4 * np.array([[x11_pred  + matrix[1,0] + matrix[2,0] +   x41_pred],
                              [matrix[0,1] + matrix[1,1] + matrix[2,1] +   x42_pred],
                              [matrix[0,2] + matrix[1,2] + matrix[2,2] + matrix[3,2]]])
    T = np.zeros((3,3))
    T[0,0] = x11_2_pred + pow(matrix[1,0],2) + pow(matrix[2,0],2) + x41_2_pred
    T[0,1] = x11_x12_pred + matrix[1,0]*matrix[1,1] + matrix[2,0]*matrix[2,1] + x41_x42_pred
    T[0,2] = x11_x13_pred + matrix[1,0]*matrix[1,2] + matrix[2,0]*matrix[2,2] + x41_x43_pred
    T[1,0] = T[0,1]
    T[1,1] = pow(matrix[0,1],2) + pow(matrix[1,1],2)+ pow(matrix[2,1],2) + x42_2_pred
    T[1,2] = matrix[0,1]*matrix[0,2] + matrix[1,1]*matrix[1,2]+ matrix[2,1]*matrix[2,2] +x42_x43_pred
    T[2,0] = T[0,2]
    T[2,1] = T[1,2]
    T[2,2] = pow(matrix[0,2],2) + pow(matrix[1,2],2) + pow(matrix[2,2],2) + pow(matrix[3,2],2)
    cov_new = 1/4*T - mean_new@mean_new.T
    m = m +1

In [15]:
m

14

In [16]:
matrix[0,0] = x11_pred
matrix[3,0] = x41_pred
matrix[3,1] = x42_pred 
matrix

array([[5.68066557, 0.        , 3.        ],
       [7.        , 2.        , 6.        ],
       [5.        , 1.        , 2.        ],
       [6.54169197, 1.46153827, 5.        ]])

In [17]:
np.linalg.norm(cov_new - cov_matrix)

6.3072377997368585e-06

In [18]:
np.linalg.norm(mean_new - mean_values.reshape(-1, 1)) 

9.518816450693215e-08